## Take Action: Select Day to analyze, and environment

In [ ]:
day_to_analyze = "2020-11-3"
env = "PAPER"  # values may be PAPER / PROD / BACKTEST

#### Imports

In [ ]:
import asyncio
import json
import math
import sys
from datetime import date, datetime, timedelta

import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics import analysis
from pytz import timezone

%matplotlib inline


nest_asyncio.apply()

#### Load trading day details

In [ ]:
day_to_analyze = datetime.strptime(day_to_analyze, "%Y-%m-%d")
trades = analysis.load_trades(day_to_analyze, env)
algo_runs = analysis.load_runs(day_to_analyze, env)

## Show trading session performance

In [ ]:
symbol_name = []
counts = []
revenues = []
est = pytz.timezone("US/Eastern")
batch_ids = trades.batch_id.unique().tolist()
# batch_ids.reverse()

current_max = pd.options.display.max_rows
pd.set_option("display.max_rows", None)
for batch_id in batch_ids:
    how_was_my_day = pd.DataFrame()
    how_was_my_day["symbol"] = trades.loc[trades["batch_id"] == batch_id][
        "symbol"
    ].unique()
    how_was_my_day["revenues"] = how_was_my_day["symbol"].apply(
        lambda x: analysis.calc_batch_revenue(x, trades, batch_id)
    )
    how_was_my_day["count"] = how_was_my_day["symbol"].apply(
        lambda x: analysis.count_trades(x, trades, batch_id)
    )

    if len(algo_runs.loc[algo_runs["batch_id"] == batch_id].start_time) > 0:
        batch_time = (
            algo_runs.loc[algo_runs["batch_id"] == batch_id]
            .start_time.min()
            .tz_localize("utc")
            .astimezone(est)
        )
    else:
        continue

    env = algo_runs[algo_runs["batch_id"] == batch_id].algo_env.tolist()[0]
    win_ratio = round(
        1.0
        * len(how_was_my_day[how_was_my_day.revenues >= 0])
        / len(how_was_my_day[how_was_my_day.revenues < 0]),
        2,
    )
    revenues = round(sum(how_was_my_day["revenues"]), 2)
    traded = len(how_was_my_day)
    print(
        f"[{env}] {batch_id}\n{batch_time}\nTotal revenues=${revenues}\nTotal traded:{traded} Win/Lose ration {win_ratio}"
    )

    display(
        Markdown(f"{len(how_was_my_day[how_was_my_day.revenues >= 0])} **Winners**")
    )
    display(how_was_my_day[how_was_my_day.revenues >= 0].sort_values('revenues'))
    display(Markdown(f"{len(how_was_my_day[how_was_my_day.revenues < 0])} **Lossers**"))
    display(how_was_my_day[how_was_my_day.revenues < 0].sort_values('revenues'))

pd.set_option("display.max_rows", current_max)

#### Load stock OHLC details

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
minute_history = {}
for symbol in trades.symbol.unique().tolist():
    minute_history[symbol] = api.polygon.historic_agg_v2(
        symbol,
        1,
        "minute",
        _from=str((day_to_analyze - timedelta(days=10)).date()),
        to=str((day_to_analyze + timedelta(days=1)).date()),
    ).df

## Show Trading details per Symbol

In [ ]:
est = pytz.timezone("US/Eastern")
position = {}
for symbol in minute_history:
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["tstamp"].min().to_pydatetime()
    start_date = start_date.replace(
        hour=9, minute=30, second=0, microsecond=0, tzinfo=None
    )
    end_date = start_date.replace(hour=16, minute=0, tzinfo=None)
    cool_down_date = start_date + timedelta(minutes=5)
    try:
        start_index = minute_history[symbol]["close"].index.get_loc(
            start_date, method="nearest"
        )
        end_index = minute_history[symbol]["close"].index.get_loc(
            end_date, method="nearest"
        )
    except Exception as e:
        print(f"Error for {symbol}: {e}")
        continue

    cool_minute_history_index = minute_history[symbol]["close"].index.get_loc(
        cool_down_date, method="nearest"
    )
    open_price = float(minute_history[symbol]["close"][cool_minute_history_index])
    plt.plot(
        minute_history[symbol]["close"][start_index:end_index],
        label=symbol,
    )
    plt.xticks(rotation=45)

    delta = 0
    profit = 0

    operations = []
    deltas = []
    profits = []
    times = []
    prices = []
    qtys = []
    indicators = []
    target_price = []
    stop_price = []
    daily_change = []
    precent_vwap = []
    algo_names = []
    for index, row in symbol_df.iterrows():

        try:
            now = int(row["client_time"])
            continue
        except Exception:
            pass

        delta = (
            row["price"]
            * row["qty"]
            * (1 if row["operation"] == "sell" and row["qty"] > 0 else -1)
        )

        profit += delta
        plt.scatter(
            row["tstamp"].to_pydatetime(),
            row["price"],
            c="g" if row["operation"] == "buy" else "r",
            s=100,
        )
        algo_names.append(row["algo_name"])
        deltas.append(round(delta, 2))
        profits.append(round(profit, 2))
        operations.append(row["operation"])
        times.append(pytz.utc.localize(pd.to_datetime(row["tstamp"])).astimezone(est))
        prices.append(row["price"])
        qtys.append(row["qty"])
        indicator = json.loads(row.indicators)
        indicators.append(indicator)
        target_price.append(row["target_price"])
        stop_price.append(row["stop_price"])
        daily_change.append(
            f"{round(100.0 * (float(row['price']) - open_price) / open_price, 2)}%"
        )
        precent_vwap.append(
            f"{round(100.0 * (indicator['buy']['avg'] - open_price) / open_price, 2)}%"
            if indicator["buy"] and "avg" in indicator["buy"]
            else ""
        )

    d = {
        "profit": profits,
        "algo": algo_names,
        "trade": deltas,
        "operation": operations,
        "at": times,
        "price": prices,
        "qty": qtys,
        "daily change": daily_change,
        "vwap": precent_vwap,
        "indicators": indicators,
        "target price": target_price,
        "stop price": stop_price,
    }
    display(Markdown("***"))
    display(Markdown(f"**{symbol}** analysis with profit **${round(profit, 2)}**"))
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()